In [17]:
import os
import re
import pandas as pd
from pdfminer.high_level import extract_text
from ipywidgets import interact, widgets, IntProgress, Label
from IPython.display import display, clear_output

# Đoạn code để xuất ra file excel sau khi đếm keyword

In [26]:
# Hàm để đếm số lần xuất hiện của các từ khóa
def count_word_occurrences(file_path, target_words, progress_bar, status_label):
    # Đọc nội dung từ file PDF
    with open(file_path, 'rb') as file:
        text = extract_text(file)

    # Chuyển nội dung thành chữ thường
    content_lower = text.lower()

    # Tạo danh sách kết quả
    occurrences = {}
    for i, target_word in enumerate(target_words):
        # Cập nhật thanh tiến trình và trạng thái
        progress_bar.value = (i + 1) / len(target_words)
        status_label.value = f"Đang kiểm tra từ khóa '{target_word}'..."
        
        # Chuyển từ khóa thành chữ thường và tìm kiếm
        target_word_lower = target_word.lower()
        pattern = re.compile(r'\b{}\b'.format(re.escape(target_word_lower)))
        count = len(re.findall(pattern, content_lower))
        occurrences[target_word] = count
        
    return occurrences

# Tự động lấy danh sách file PDF trong thư mục
directory_path = "DemoPDF/"  # Thư mục chứa các file PDF
pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]

# Đọc danh sách từ khóa từ file txt
txt_file = "keywords.txt"
with open(txt_file, "r", encoding="utf-8") as file:
    keywords = [line.strip() for line in file.readlines()]

# Thiết lập giao diện
pdf_file_dropdown = widgets.Dropdown(options=pdf_files, description='Select PDF File')
submit_button = widgets.Button(description='Submit')
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progress:', bar_style='info')
status_label = widgets.Label(value="")

result_output = widgets.Output()

# Hàm xử lý khi bấm nút Submit
def on_submit_clicked(b):
    pdf_file = pdf_file_dropdown.value
    progress_bar.value = 0
    status_label.value = "Bắt đầu đếm từ khóa..."
    count_results = count_word_occurrences(os.path.join(directory_path, pdf_file), keywords, progress_bar, status_label)
    
    # Hiển thị kết quả và chuẩn bị dữ liệu cho file Excel
    new_results = [{"PDF File": pdf_file, "Keyword": keyword, "Count": count} for keyword, count in count_results.items()]
    new_df = pd.DataFrame(new_results)
    
    output_file = "all_pdf_keyword_counts.xlsx"
    
    # Nếu file Excel đã tồn tại, thêm dữ liệu mới vào sau các dữ liệu cũ
    if os.path.exists(output_file):
        existing_df = pd.read_excel(output_file)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        combined_df.to_excel(output_file, index=False)
    else:
        # Nếu file chưa tồn tại, tạo file mới
        new_df.to_excel(output_file, index=False)
    
    with result_output:
        result_output.clear_output()
        print(f"Kết quả đã được lưu vào file Excel: {output_file}")
    
    status_label.value = "Hoàn thành!"

# Gán hàm xử lý vào nút
submit_button.on_click(on_submit_clicked)

# Hiển thị giao diện
display(pdf_file_dropdown, submit_button, progress_bar, status_label, result_output)

Dropdown(description='Select PDF File', options=('CAG_Baocaothuongnien_2018.pdf', 'CMP_Baocaothuongnien_2021.p…

Button(description='Submit', style=ButtonStyle())

FloatProgress(value=0.0, bar_style='info', description='Progress:', max=1.0)

Label(value='')

Output()

# Đoạn code để xuất ra file json sang sau khi đếm keyword

In [ ]:
import json
import os
import re
from pdfminer.high_level import extract_text
import ipywidgets as widgets
from IPython.display import display

# Hàm để đếm số lần xuất hiện của các từ khóa với phân biệt hoa-thường
def count_word_occurrences(file_path, target_words, progress_bar, status_label):
    with open(file_path, 'rb') as file:
        text = extract_text(file)
    
    # Loại bỏ các ngắt dòng và giữ lại khoảng trắng
    text = re.sub(r'-\n', '', text)  # Loại bỏ ngắt dòng khi từ bị nối bằng dấu gạch ngang
    text = re.sub(r'\n+', ' ', text)  # Thay thế các ngắt dòng liên tục bằng khoảng trắng
    
    occurrences = {}
    for i, target_word in enumerate(target_words):
        progress_bar.value = (i + 1) / len(target_words)
        status_label.value = f"Đang kiểm tra từ khóa '{target_word}'..."
        
        # Kiểm tra phân biệt hoa-thường cho "AI" và "ai"
        if target_word == "AI":
            count = len(re.findall(r'\bAI\b', text))
        elif target_word == "ai":
            count = len(re.findall(r'\bai\b', text))
        else:
            pattern = re.compile(r'\b{}\b'.format(re.escape(target_word)), re.IGNORECASE)
            count = len(re.findall(pattern, text))
        
        occurrences[target_word] = count

    return occurrences



# Tự động lấy danh sách file PDF trong thư mục
directory_path = "DemoPDF/"  
pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]

# Đọc danh sách từ khóa từ file txt
txt_file = "keywords.txt"
with open(txt_file, "r", encoding="utf-8") as file:
    keywords = [line.strip() for line in file.readlines()]

# Thiết lập giao diện với "All PDF Files" ở đầu danh sách
pdf_file_dropdown = widgets.Dropdown(
    options=["All PDF Files"] + pdf_files,  
    description='Select PDF File'
)
submit_button = widgets.Button(description='Submit')
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progress:', bar_style='info')
status_label = widgets.Label(value="")
result_output = widgets.Output()

# Hàm xử lý khi bấm nút Submit
def on_submit_clicked(b):
    selected_option = pdf_file_dropdown.value
    output_file = "all_pdf_keyword_counts.json"
    all_results = []

    if selected_option == "All PDF Files":
        status_label.value = "Bắt đầu đếm từ khóa cho tất cả các tệp PDF..."
        for pdf_file in pdf_files:
            count_results = count_word_occurrences(os.path.join(directory_path, pdf_file), keywords, progress_bar, status_label)
            new_results = [{"PDF File": pdf_file, "Keyword": keyword, "Count": count} for keyword, count in count_results.items()]
            all_results.extend(new_results)
    else:
        pdf_file = selected_option
        status_label.value = f"Đang đếm từ khóa cho tệp: {pdf_file}"
        count_results = count_word_occurrences(os.path.join(directory_path, pdf_file), keywords, progress_bar, status_label)
        all_results = [{"PDF File": pdf_file, "Keyword": keyword, "Count": count} for keyword, count in count_results.items()]

    # Ghi kết quả vào JSON
    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        with open(output_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
            existing_data.extend(all_results)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(existing_data, f, ensure_ascii=False, indent=4)
    else:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, ensure_ascii=False, indent=4)
    
    with result_output:
        result_output.clear_output()
        print(f"Kết quả đã được lưu vào file JSON: {output_file}")
    
    status_label.value = "Hoàn thành!"

# Gán hàm xử lý vào nút
submit_button.on_click(on_submit_clicked)

# Hiển thị giao diện
display(pdf_file_dropdown, submit_button, progress_bar, status_label, result_output)


Dropdown(description='Select PDF File', options=('All PDF Files', 'CAG_Baocaothuongnien_2018.pdf', 'CMP_Baocao…

Button(description='Submit', style=ButtonStyle())

FloatProgress(value=0.0, bar_style='info', description='Progress:', max=1.0)

Label(value='')

Output()

# Update code phân loại encode và không encode

In [ ]:
import json
import os
import re
import shutil
from pdfminer.high_level import extract_text
import ipywidgets as widgets
from IPython.display import display

# Hàm để đếm số lần xuất hiện của các từ khóa với phân biệt hoa-thường
def count_word_occurrences(file_path, target_words, progress_bar, status_label):
    with open(file_path, 'rb') as file:
        text = extract_text(file)

    occurrences = {}
    total_count = 0  # Tổng số lần từ khóa xuất hiện
    
    for i, target_word in enumerate(target_words):
        progress_bar.value = (i + 1) / len(target_words)
        status_label.value = f"Đang kiểm tra từ khóa '{target_word}'..."
        
        # Kiểm tra phân biệt hoa-thường cho "AI" và "ai"
        if target_word == "AI":
            count = len(re.findall(r'\bAI\b', text))
        elif target_word == "ai":
            count = len(re.findall(r'\bai\b', text))
        else:
            pattern = re.compile(r'\b{}\b'.format(re.escape(target_word)), re.IGNORECASE)
            count = len(re.findall(pattern, text))
        
        occurrences[target_word] = count
        total_count += count  # Cộng dồn số lần xuất hiện cho từng từ khóa

    # Kiểm tra phân loại PDF
    is__non_encoded = total_count > 0  # PDF được mã hóa nếu có ít nhất 1 từ khóa xuất hiện
    return occurrences, is__non_encoded

# Tự động lấy danh sách file PDF trong thư mục
directory_path = "DemoScan/non_scanned_pdfs/"
pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]

# Đọc danh sách từ khóa từ file txt
txt_file = "keywords.txt"
with open(txt_file, "r", encoding="utf-8") as file:
    keywords = [line.strip() for line in file.readlines()]

# Thiết lập giao diện với "All PDF Files" ở đầu danh sách
pdf_file_dropdown = widgets.Dropdown(
    options=["All PDF Files"] + pdf_files,
    description='Select PDF File'
)
submit_button = widgets.Button(description='Submit')
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progress:', bar_style='info')
status_label = widgets.Label(value="")
result_output = widgets.Output()

# Đường dẫn thư mục đích để lưu các file đã phân loại
encoded_dir = "DemoScan/non_scanned_pdfs/Encoded_PDFs"
non_encoded_dir = "DemoScan/non_scanned_pdfs/Non_Encoded_PDFs"
os.makedirs(encoded_dir, exist_ok=True)
os.makedirs(non_encoded_dir, exist_ok=True)

# Hàm xử lý khi bấm nút Submit
def on_submit_clicked(b):
    selected_option = pdf_file_dropdown.value
    output_file = "all_pdf_keyword_counts.json"
    all_results = []

    if selected_option == "All PDF Files":
        status_label.value = "Bắt đầu đếm từ khóa cho tất cả các tệp PDF..."
        for pdf_file in pdf_files:
            count_results, is__non_encoded = count_word_occurrences(os.path.join(directory_path, pdf_file), keywords, progress_bar, status_label)
            new_results = [{"PDF File": pdf_file, "Keyword": keyword, "Count": count} for keyword, count in count_results.items()]
            all_results.extend(new_results)
            
            # Di chuyển file vào thư mục đã phân loại
            if is__non_encoded:
                shutil.move(os.path.join(directory_path, pdf_file), os.path.join(non_encoded_dir, pdf_file))
            else:
                shutil.move(os.path.join(directory_path, pdf_file), os.path.join(encoded_dir, pdf_file))
                
    else:
        pdf_file = selected_option
        status_label.value = f"Đang đếm từ khóa cho tệp: {pdf_file}"
        count_results, is__non_encoded = count_word_occurrences(os.path.join(directory_path, pdf_file), keywords, progress_bar, status_label)
        all_results = [{"PDF File": pdf_file, "Keyword": keyword, "Count": count} for keyword, count in count_results.items()]

        # Di chuyển file vào thư mục đã phân loại
        if is__non_encoded:
            shutil.move(os.path.join(directory_path, pdf_file), os.path.join(non_encoded_dir, pdf_file))
        else:
            shutil.move(os.path.join(directory_path, pdf_file), os.path.join(encoded_dir, pdf_file))

    # Ghi kết quả đếm từ khóa vào JSON
    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        with open(output_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
            existing_data.extend(all_results)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(existing_data, f, ensure_ascii=False, indent=4)
    else:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, ensure_ascii=False, indent=4)

    with result_output:
        result_output.clear_output()
        print(f"Kết quả đã được lưu vào file JSON: {output_file}")
    
    status_label.value = "Hoàn thành!"

# Gán hàm xử lý vào nút
submit_button.on_click(on_submit_clicked)

# Hiển thị giao diện
display(pdf_file_dropdown, submit_button, progress_bar, status_label, result_output)


Dropdown(description='Select PDF File', options=('All PDF Files', 'ACV_Baocaothuongnien_2019.pdf', 'BC.pdf'), …

Button(description='Submit', style=ButtonStyle())

FloatProgress(value=0.0, bar_style='info', description='Progress:', max=1.0)

Label(value='')

Output()

# Đoạn code chuyển dữ liệu từ file json sang excel

In [45]:
import pandas as pd
import json

# Đọc dữ liệu từ file JSON
json_file = "all_pdf_keyword_counts.json"  # Đường dẫn tới file JSON
with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Chuyển đổi dữ liệu thành DataFrame
df = pd.DataFrame(data)

# Lưu DataFrame vào file Excel
excel_file = "all_pdf_keyword_counts.xlsx"  # Đường dẫn tới file Excel đầu ra
df.to_excel(excel_file, index=False)

print(f"Dữ liệu đã được chuyển đổi và lưu vào file Excel: {excel_file}")


Dữ liệu đã được chuyển đổi và lưu vào file Excel: all_pdf_keyword_counts.xlsx


# Đoạn code phân loại File scan và không scan

In [43]:
import os
import shutil
import fitz
import time

def get_text_percentage(file_name: str) -> float:
    """
    Calculate the percentage of document that is covered by (searchable) text.
    
    If the returned percentage of text is very low, the document is
    most likely a scanned PDF.
    """
    total_page_area = 0.0
    total_text_area = 0.0

    doc = fitz.open(file_name)

    for page in doc:
        total_page_area += abs(page.rect)  # Diện tích tổng của trang
        text_area = sum(abs(fitz.Rect(b[:4])) for b in page.get_text("blocks"))  # Tính tổng diện tích văn bản
        total_text_area += text_area

    doc.close()
    return total_text_area / total_page_area if total_page_area > 0 else 0

def classify_pdfs(source_directory: str, scanned_folder: str, non_scanned_folder: str):
    """
    Classify PDFs into scanned and non-scanned folders based on text content percentage.
    """
    # Tạo các thư mục cho file scan và không scan nếu chưa tồn tại
    os.makedirs(scanned_folder, exist_ok=True)
    os.makedirs(non_scanned_folder, exist_ok=True)

    # Lấy danh sách các file PDF trong thư mục nguồn
    pdf_files = [f for f in os.listdir(source_directory) if f.endswith('.pdf')]

    # Phân loại và di chuyển các file PDF
    for pdf_file in pdf_files:
        file_path = os.path.join(source_directory, pdf_file)

        # Tính phần trăm văn bản
        text_perc = get_text_percentage(file_path)
        
        # In ra phần trăm văn bản
        print(f"{pdf_file}: {text_perc:.2%} text present")

        # Định nghĩa thư mục đích
        destination = scanned_folder if text_perc < 0.01 else non_scanned_folder

        # Thử di chuyển file, xử lý lỗi nếu cần
        max_retries = 5  # Số lần thử tối đa
        retries = 0
        while retries < max_retries:
            try:
                shutil.move(file_path, os.path.join(destination, pdf_file))
                print(f"Moved {pdf_file} to {destination}.")
                break  # Thoát khỏi vòng lặp nếu di chuyển thành công
            except PermissionError:
                retries += 1
                print(f"File {pdf_file} is currently in use. Retrying ({retries}/{max_retries}) in 1 second...")
                time.sleep(1)  # Chờ 1 giây trước khi thử lại
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                break  # Thoát khỏi vòng lặp nếu có lỗi khác

if __name__ == "__main__":
    source_directory = "./DemoScan"  # Đường dẫn đến thư mục chứa các file PDF
    scanned_folder = "./DemoScan/scanned_pdfs"
    non_scanned_folder = "./DemoScan/non_scanned_pdfs"
    
    classify_pdfs(source_directory, scanned_folder, non_scanned_folder)


ACV_Baocaothuongnien_2019.pdf: 42.64% text present
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (1/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (2/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (3/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (4/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (5/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (6/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (7/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (8/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (9/10) in 1 second...
File ACV_Baocaothuongnien_2019.pdf is currently in use. Retrying (10/10) in 1 second...
DNL_Baocaothuongnien_2020.pdf: 0.00% text present
Moved DNL_Baocaothuongnien_2

# Đoạn code lấy ảnh trong thư mục và dùng i2OCR

In [ ]:
import selenium
print("Version " + selenium.__version__)

# <span class="btn btn-file"><span class="fileupload-new">Select Image</span><span class="fileupload-exists">Change</span><input type="file" id="i2ocr_uploadedfile" name="i2ocr_uploadedfile"></span>
# <input type="submit" style="margin-left:-50px;" title="Perform OCR" class="btn btn-large btn-danger bold space" id="submit_i2ocr" value="Extract Text">
# <textarea id="ocrTextBox" rows="12" class="i2ocr_iTextArea" style="display: inline-block; direction: ltr;"></textarea>

# Xpath, //div/span[@class='btn btn-file']

Version 4.25.0


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

In [ ]:
# Demo
from selenium import webdriver
from selenium.webdriver.common.by import By
from pynput.keyboard import Key, Controller
import time
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get("https://2ocr.com/online-ocr-vietnamese/")
driver.find_element(By.XPATH, "//div[@class='filepicker dropzone filePicker dz-clickable']").click()
time.sleep(3)

keyboard = Controller()
keyboard.type("E:\\NCKHDigital\\Photos\\BCphotos\\0004.jpg")
keyboard.press(Key.enter)


# driver.quit()

In [ ]:
# https://www.i2ocr.com/free-online-vietnamese-ocr
from selenium import webdriver
from selenium.webdriver.common.by import By
from pynput.keyboard import Key, Controller
import time
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get("https://www.i2ocr.com/free-online-vietnamese-ocr/")
driver.find_element(By.XPATH, "//div/span[@class='btn btn-file']").click()
time.sleep(3)

keyboard = Controller()
keyboard.type("E:\\NCKHDigital\\Photos\\BCphotos\\0004.jpg")
keyboard.press(Key.enter)


In [23]:
# Đường dẫn tới ảnh bạn muốn upload và file txt lưu trữ kết quả
image_path = './Photos/BCphotos/0004.jpg'
output_file_path = 'output.txt'

# Khởi tạo trình duyệt và truy cập vào trang web
driver = webdriver.Chrome()  # Hoặc ChromeDriver, GeckoDriver tùy thuộc vào trình duyệt
driver.get("https://www.i2ocr.com/free-online-vietnamese-ocr")

try:
    # 1. Tìm nút "Select Image" và tải ảnh lên
    upload_button = driver.find_element(By.CLASS_NAME, "btn-file")
    file_input = upload_button.find_element(By.TAG_NAME, "input")
    file_input.send_keys(image_path)
    
    # 2. Tìm nút "Extract Text" và bấm vào
    submit_button = driver.find_element(By.ID, "submit_i2ocr")
    submit_button.click()
    
    # Chờ để đảm bảo OCR hoàn tất
    time.sleep(10)  # Thời gian có thể thay đổi tùy thuộc vào tốc độ xử lý
    
    # 3. Lấy văn bản từ ô text
    ocr_textbox = driver.find_element(By.ID, "ocrTextBox")
    ocr_text = ocr_textbox.get_attribute("value")
    
    # Ghi kết quả vào file .txt
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write(ocr_text)

finally:
    # Đóng trình duyệt sau khi hoàn thành
    driver.quit()


WebDriverException: Message: unknown error: path is not absolute: ./Photos/BCphotos/0004.jpg
  (Session info: chrome=130.0.6723.70)
Stacktrace:
	GetHandleVerifier [0x00007FF7B8003AF5+28005]
	(No symbol) [0x00007FF7B7F683F0]
	(No symbol) [0x00007FF7B7E0580A]
	(No symbol) [0x00007FF7B7E1567C]
	(No symbol) [0x00007FF7B7E4D4A0]
	(No symbol) [0x00007FF7B7E7BA3A]
	(No symbol) [0x00007FF7B7E49246]
	(No symbol) [0x00007FF7B7E7BC50]
	(No symbol) [0x00007FF7B7E9B8B3]
	(No symbol) [0x00007FF7B7E7B7E3]
	(No symbol) [0x00007FF7B7E475C8]
	(No symbol) [0x00007FF7B7E48731]
	GetHandleVerifier [0x00007FF7B82F646D+3118813]
	GetHandleVerifier [0x00007FF7B8346CC0+3448624]
	GetHandleVerifier [0x00007FF7B833CF3D+3408301]
	GetHandleVerifier [0x00007FF7B80CA44B+841403]
	(No symbol) [0x00007FF7B7F7344F]
	(No symbol) [0x00007FF7B7F6F4C4]
	(No symbol) [0x00007FF7B7F6F65D]
	(No symbol) [0x00007FF7B7F5EBB9]
	BaseThreadInitThunk [0x00007FFCBE027374+20]
	RtlUserThreadStart [0x00007FFCBE1FCC91+33]
